### DATA ENGINEERING PLATFORMS (MSCA 31012)
### File        :   Class Exercise - Session 8 - PythonMongoClient
### Desc     :  Connecting to MongoDB via Jupyter Notebook                  

References: 
https://docs.mongodb.com/getting-started/python/client/
https://blog.exploratory.io/an-introduction-to-mongodb-query-for-beginners-bd463319aa4c/
https://docs.mongodb.com/manual/tutorial/getting-started/ 

Installation:
`pip install pymongo`

pymongo is an interface for connecting to a Mongo database server from Python. The steps are as follows:

1. Install and start MongoDB on your local machine.
2. Make sure to run mongod with the data folder option
    "C:\Program Files\MongoDB\Server\4.2\bin\mongod.exe" --dbpath "C:\data"
    
    Mac Users : /usr/local/mongoDB/bin/mongod --dbpath "C:\data"

3. Download file 
https://raw.githubusercontent.com/mongodb/docs-assets/primer-dataset/primer-dataset.json

4. Import downloaded sample data into Mongo DB (25359 documents)
"C:\Program Files\MongoDB\Server\4.2\bin\mongoimport" --db test --collection restaurants --drop --file "C:\Users\SBharadwaj\Desktop\Shree\DEPA\03-Lectures\8\Exercises\MongoDB\primer-dataset.json"

MAC : mongoimport --db test --collection restaurants --drop --file "C:\Users\SBharadwaj\Desktop\Shree\DEPA\03-Lectures\8\Exercises\MongoDB\primer-dataset.json"

In [1]:
#Installation:
!pip install pymongo

   ---------------------------------------- 0.0/949.4 kB ? eta -:--:--
   ----------- ---------------------------- 262.1/949.4 kB ? eta -:--:--
   ---------------------------------------- 949.4/949.4 kB 4.6 MB/s eta 0:00:00

   ---------------------------------------- 0/2 [dnspython]
   ---------------------------------------- 0/2 [dnspython]
   ---------------------------------------- 0/2 [dnspython]
   ---------------------------------------- 0/2 [dnspython]
   ---------------------------------------- 0/2 [dnspython]
   ---------------------------------------- 0/2 [dnspython]
   ---------------------------------------- 0/2 [dnspython]
   ---------------------------------------- 0/2 [dnspython]
   ---------------------------------------- 0/2 [dnspython]
   ---------------------------------------- 0/2 [dnspython]
   ---------------------------------------- 0/2 [dnspython]
   ---------------------------------------- 0/2 [dnspython]
   ---------------------------------------- 0/2 [dnspyt

In [2]:
import pymongo
import json
from pymongo import MongoClient

### Connect to MongoDB

In [3]:
#connect to local database server
client = MongoClient()

#switch to test DB
db = client.test

In [4]:
def printhead(cursor, n):
    for idx,document in enumerate(cursor):
        if idx <= n: 
            print(document)
        else:
            break

### Query MongoDB

In [5]:
restaurants = db.restaurants.find()
printhead(restaurants, 2)

#### Cut paste the JSON results into a JSON formatter ( URL below ) and click on format to get a clean view of the data
http://jsonviewer.stack.hu/  ( Pretty JSON ) 

In [6]:
# List all documents in the restaurant collection where borough is Manhattan
restaurantData = db.restaurants.find({"borough": "Manhattan"})
printhead(restaurantData, 2)

In [7]:
#### Sort the query results based on borougn and zipcode
restaurantData = db.restaurants.find().sort([
    ("borough", pymongo.ASCENDING),
    ("address.zipcode", pymongo.ASCENDING)
])
printhead(restaurantData, 2)

#### Insert data

In [8]:
from datetime import datetime
result = db.restaurants.insert_one(
    {
        "address": {
            "street": "2 Avenue",
            "zipcode": "10075",
            "building": "1480",
            "coord": [-73.9557413, 40.7720266]
        },
        "borough": "Manhattan",
        "cuisine": "Italian",
        "grades": [
            {
                "date": datetime.strptime("2014-10-01", "%Y-%m-%d"),
                "grade": "A",
                "score": 11
            },
            {
                "date": datetime.strptime("2014-01-16", "%Y-%m-%d"),
                "grade": "B",
                "score": 17
            }
        ],
        "name": "Vella",
        "restaurant_id": "41704620"
    }
)

In [9]:
result

InsertOneResult(ObjectId('6838d8e7f6d13209d86d79db'), acknowledged=True)

In [10]:
#check document that was inserted
result.inserted_id

ObjectId('6838d8e7f6d13209d86d79db')

#### Insert more than one documents

In [11]:
result = db.test.insert_many([{
        "address": {
            "street": "2 Avenue",
            "zipcode": "10075",
            "building": "1480",
            "coord": [-72.937413, 40.75466]
        },
        "borough": "Manhattan",
        "cuisine": "Indian",
        "grades": [
            {
                "date": datetime.strptime("2014-10-01", "%Y-%m-%d"),
                "grade": "A",
                "score": 11
            },
            {
                "date": datetime.strptime("2015-05-16", "%Y-%m-%d"),
                "grade": "B",
                "score": 17
            }
        ],
        "name": "India Garden",
        "restaurant_id": "4170462" + str(i)
    } for i in range(4)])

In [13]:
result.inserted_ids

[ObjectId('6838d8f5f6d13209d86d79dc'),
 ObjectId('6838d8f5f6d13209d86d79dd'),
 ObjectId('6838d8f5f6d13209d86d79de'),
 ObjectId('6838d8f5f6d13209d86d79df')]

#### Update documents { update_one(),update_many() }
In MongoDB, a write operation is atomic on the level of a single document, even if the operation modifies multiple embedded documents within a single document. When a single write operation modifies multiple documents, the modification of each document is atomic, but the operation as a whole is not atomic and other operations may interleave.However, you can isolate a single write operation that affects multiple documents using the $isolated operator.

In [15]:
result = db.restaurants.update_one(
    {"cuisine": "Indian"},
    {
        "$set": {
            "name": "Mexican Garden"
        },
        "$currentDate": {"lastModified": True}
    }
)

In [16]:
print (result.matched_count)
print (result.modified_count)
cursor = db.restaurants.find({"name": "Mexican Garden"})
printhead(cursor, 10)

0
0


####  Replace documents {replace_one()}
To replace the entire document rather than selected fields

In [17]:
result = db.restaurants.replace_one(
    {"restaurant_id": "41704620"},
    {
        "name": "Mexican Garden",
        "cuisine": "Mexican",
        "address": {
            "coord": [-73.9557413, 40.7720266],
            "building": "1480",
            "street": "2 Avenue",
            "zipcode": "10075"
        }
    }
)

In [18]:
print (result.matched_count)
print (result.modified_count)
cursor = db.restaurants.find({"name": "Mexican Garden"})
printhead(cursor, 10)

1
1
{'_id': ObjectId('6838d8e7f6d13209d86d79db'), 'name': 'Mexican Garden', 'cuisine': 'Mexican', 'address': {'coord': [-73.9557413, 40.7720266], 'building': '1480', 'street': '2 Avenue', 'zipcode': '10075'}}


####  Data Aggregation, Grouping & Sorting 
Documents enter a multi-stage pipeline that transforms the documents into aggregated results

In [19]:
# Groups documents by city and get counts of each sorted in descending order
cursor = db.restaurants.aggregate(
    [ 
        { '$group': { '_id': "$borough", "count": { '$sum': 1 } } },
        { '$sort' : {'count' : -1} }
    ]
)
printhead(cursor, 10)

{'_id': None, 'count': 1}


In [20]:
# find a list of restaurants located in the Bronx, grouped by restaurant category
cursor = db.restaurants.aggregate( 
      [ 
          { '$match': { "borough": "Bronx" } },
#           { '$unwind': '$categories'},
          { '$group': { '_id': "$categories", 'Bronx restaurants': { '$sum': 1 } } }
      ]  )
printhead(cursor, 10)

In [21]:
# The following pipeline uses $match to query the restaurants collection for documents with borough 
# equal to "Queens" and cuisine equal to Brazilian. The _id field contains the distinct zipcode value.
cursor = db.restaurants.aggregate(
   [
     { '$match': { "borough": "Queens", "cuisine": "Brazilian" } },
     { '$group': { "_id": "$address.zipcode" , "count": { '$sum': 1 } } }
   ] )
printhead(cursor, 10)

### Delete documents

delete_one(), delete_many()

In [22]:
#delete one document where name matches condition
db.restaurants.delete_one({"name": "India Garden"})

DeleteResult({'n': 0, 'ok': 1.0}, acknowledged=True)

In [23]:
#delete all documents where name matches condition
db.restaurants.delete_many({"name": "India Garden"})

DeleteResult({'n': 0, 'ok': 1.0}, acknowledged=True)

In [24]:
#delete all documents - empties the DB
db.restaurants.delete_many({})

DeleteResult({'n': 1, 'ok': 1.0}, acknowledged=True)

In [25]:
#find the first 10 documents in the db - none should be found since the data was deleted 
cursor = db.restaurants.find()
printhead(cursor, 10)

### Explore Further

https://docs.mongodb.com/manual/core/bulk-write-operations/

https://docs.mongodb.com/manual/reference/sql-comparison/